In [1]:
import numpy as np
import pandas as pd

import os

hasModel = False
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        if(filename == 'diabete_model.pkl'):
            print('Already have diabete_model.pkl')
            hasModel = True

Already have diabete_model.pkl


In [2]:
df=pd.read_csv("./dataset/Diabetes.csv")

In [3]:
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [4]:
df.describe()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,41.885856,0.07485,0.039420,27.320767,5.527507,138.058060,0.085000
std,22.516840,0.26315,0.194593,6.636783,1.070672,40.708136,0.278883
min,0.080000,0.00000,0.000000,10.010000,3.500000,80.000000,0.000000
25%,24.000000,0.00000,0.000000,23.630000,4.800000,100.000000,0.000000
50%,43.000000,0.00000,0.000000,27.320000,5.800000,140.000000,0.000000
75%,60.000000,0.00000,0.000000,29.580000,6.200000,159.000000,0.000000
max,80.000000,1.00000,1.000000,95.690000,9.000000,300.000000,1.000000


In [5]:
df = df[df['gender'] != 'Other']

df["gender"]=df['gender'].replace("Female",1).replace("Male",2)
df["smoking_history"]=df['smoking_history'].replace(['current', 'ever', 'former', 'not current'],1).replace(['never', 'No Info'],0)

# df["BMI"]=df['BMI'].replace(0,df['BMI'].mean())
# df['Insulin']=df['Insulin'].replace(0,df['Insulin'].mean())
# df['SkinThickness']=df['SkinThickness'].replace(0,df['SkinThickness'].mean())
# df['BloodPressure']=df['BloodPressure'].replace(0,df['BloodPressure'].mean())
# df['Glucose']=df['Glucose'].replace(0,df['Glucose'].mean())
# df['DiabetesPedigreeFunction']=df['DiabetesPedigreeFunction'].replace(0,df['DiabetesPedigreeFunction'].mean())

In [6]:
df.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix

import pickle

In [8]:
x=df[['age', 'gender', 'bmi', 'smoking_history']]
y=df['diabetes']

In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [10]:
log_ref=LogisticRegression(random_state = 42)

In [11]:
log_ref.fit(x_train,y_train)

LogisticRegression(random_state=42)

In [12]:
parameters={
    'penalty':['l1', 'l2', 'elasticnet', None],
    'C':np.logspace(-3,3,7),
    'solver':['newton-cg','lbfgs','liblinear']
}
import warnings
warnings.filterwarnings('ignore')

In [13]:
if not hasModel:
    clf=GridSearchCV(
        log_ref,param_grid=parameters,
        scoring='accuracy',
        cv=10
    )
    clf.fit(x_train,y_train)
else:
    with open('diabete_model.pkl','rb') as f:
        clf = pickle.load(f)

In [14]:
y_pred=clf.predict(x_test)

In [15]:
conf_mat=confusion_matrix(y_test,y_pred)
conf_mat

array([[22743,    62],
       [ 2125,    66]], dtype=int64)

In [16]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.9125060009601537

In [17]:
with open('diabete_model.pkl','wb') as f:
    pickle.dump(clf,f)

In [18]:
print('Successfully imported diabete_model')

Successfully imported diabete_model
